In [1]:
#!/usr/bin/env python
# -*- coding: utf-8 -*-
#######################
__version__ = "1.0"
__date__ = "2016-04-28"
__modified_by__ = "Hrushikesh Dhumal"
####################################

In [2]:
import os
import string
import numpy as np
import pandas as pd
from utils import *

In [3]:
from bs4 import BeautifulSoup
import re
import nltk
from nltk import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer

In [4]:
SEED_VAL = 200;
WORK_DIR = os.getcwd();
data_subset = "_10Percent"
YELP_DATA_CSV_DIR = os.path.join(WORK_DIR, "data", "csv")
YELP_DATA_WORD_2_VEC_MODEL_DIR = os.path.join(WORK_DIR, "data", "word2vec_model")

In [5]:
make_sure_path_exists(YELP_DATA_WORD_2_VEC_MODEL_DIR)

In [6]:
read_filename = os.path.join(YELP_DATA_CSV_DIR, 'business_review_user' + data_subset + '.csv')
df_data = pd.read_csv(read_filename, engine='c', encoding='utf-8')

C:\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:2723: DtypeWarning: Columns (5,9) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
df_data.review_text[1]

u"Great outdoor patio dining area. Great happy hour. Great service.\r\n\r\nOutdoor patio dining has a beautiful mesquite tree for ambiance and blocking out the sun while the center fireplace keeps you warm. \r\n\r\nWe had:\r\nQueso Skillet with warm tortilla chips - amazing!\r\nMac N Cheese with Chorizo - fabulous! one of the best mac n cheeses I've ever had!\r\nCarne Asada on a Potato Pancake - was ok. Sounded excellent, tasted decent.\r\n\r\nFriendly and good food. But the ambiance really puts it over the top as a great dining experience. I'd be back with a group of friends to lounge, play cornsack or bocce ball during happy hour."

In [8]:
def lowercase_remove_punctuation_and_numbers_and_tokenize_and_filter_more_stopwords_and_stem(s):
    s = remove_numbers_in_string(s)
    s = lowercase_remove_punctuation(s)
    s = remove_stopwords(s)
    token_list = nltk.word_tokenize(s)
    #token_list = filter_out_more_stopwords(token_list)
    token_list = stem_token_list(token_list)
    return token_list

In [9]:
print lowercase_remove_punctuation_and_numbers_and_tokenize_and_filter_more_stopwords_and_stem(df_data.review_text[1])

[u'great', u'outdoor', u'patio', u'dine', u'area', u'great', u'happi', u'hour', u'great', u'servic', u'outdoor', u'patio', u'dine', u'beauti', u'mesquit', u'tree', u'ambianc', u'block', u'sun', u'center', u'fireplac', u'keep', u'warm', u'queso', u'skillet', u'warm', u'tortilla', u'chip', u'amaz', u'mac', u'n', u'chees', u'chorizo', u'fabul', u'one', u'best', u'mac', u'n', u'chees', u'ive', u'ever', u'carn', u'asada', u'potato', u'pancak', u'ok', u'sound', u'excel', u'tast', u'decent', u'friendli', u'good', u'food', u'ambianc', u'realli', u'put', u'top', u'great', u'dine', u'experi', u'id', u'back', u'group', u'friend', u'loung', u'play', u'cornsack', u'bocc', u'ball', u'happi', u'hour']


In [10]:
# Load the punkt tokenizer
tokenizer = nltk.data.load('tokenizers/punkt/english.pickle')

# Define a function to split a review into parsed sentences
def review_to_sentences( review, tokenizer, remove_stopwords=False ):
    # Function to split a review into parsed sentences. Returns a 
    # list of sentences, where each sentence is a list of words
    #
    # 1. Use the NLTK tokenizer to split the paragraph into sentences
    raw_sentences = tokenizer.tokenize(review.strip())
    #
    # 2. Loop over each sentence
    sentences = []
    for raw_sentence in raw_sentences:
        # If a sentence is empty, skip it
        if len(raw_sentence) > 0:
            # Otherwise, call review_to_wordlist to get a list of words
            sentences.append( lowercase_remove_punctuation_and_numbers_and_tokenize_and_filter_more_stopwords_and_stem( raw_sentence))
    
    # Return the list of sentences (each sentence is a list of words,
    # so this returns a list of lists
    return sentences

In [11]:
sentences = []  # Initialize an empty list of sentences

print "Parsing sentences from training set"
for review in df_data["review_text"]:
    sentences += review_to_sentences(review, tokenizer)

Parsing sentences from training set


In [12]:
print sentences[0]

[u'locat', u'chevron', u'ga', u'station', u'lot']


In [13]:
# Import the built-in logging module and configure it so that Word2Vec 
# creates nice output messages
import gensim, logging
from gensim.models import Word2Vec
logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

Using gpu device 0: GeForce GTX 965M (CNMeM is disabled, CuDNN not available)


In [14]:
# Set values for various parameters
num_features = 300    # Word vector dimensionality                      
min_word_count = 10   # Minimum word count                        
num_workers = 8       # Number of threads to run in parallel
context = 10          # Context window size                                                                                    
downsampling = 1e-3   # Downsample setting for frequent words

# Initialize and train the model (this will take some time)
from gensim.models import word2vec

model_file = os.path.join(YELP_DATA_WORD_2_VEC_MODEL_DIR, str(num_features) + 'features_' + str(min_word_count) + 'minwords_' + str(context) + 'context'+ data_subset)
if not os.path.isfile(model_file):
    print "Training model..."
# It can be helpful to create a meaningful model name and 
# save the model for later use. You can load it later using Word2Vec.load()
    model = word2vec.Word2Vec(sentences, workers=num_workers, \
                size=num_features, min_count = min_word_count, \
                window = context, sample = downsampling)
# If you don't plan to train the model any further, calling 
# init_sims will make the model much more memory-efficient.
    model.init_sims(replace=True)
    model.save(model_file)
else:
    print "Loading existing model"
    model = Word2Vec.load(model_file)

Loading existing model


In [15]:
model.doesnt_match("amazing delightful bad".split())

'amazing'

In [16]:
def makeFeatureVec(words, model, num_features):
    # Function to average all of the word vectors in a given
    # paragraph
    #
    # Pre-initialize an empty numpy array (for speed)
    featureVec = np.zeros((num_features,),dtype="float32")
    #
    nwords = 0.
    # 
    # Index2word is a list that contains the names of the words in 
    # the model's vocabulary. Convert it to a set, for speed 
    index2word_set = set(model.index2word)
    #
    # Loop over each word in the review and, if it is in the model's
    # vocaublary, add its feature vector to the total
    for word in words:
        if word in index2word_set: 
            nwords = nwords + 1.
            featureVec = np.add(featureVec,model[word])
    # 
    # Divide the result by the number of words to get the average
    featureVec = np.divide(featureVec,nwords)
    return featureVec


def getAvgFeatureVecs(reviews, model, num_features):
    # Given a set of reviews (each one a list of words), calculate 
    # the average feature vector for each one and return a 2D numpy array 
    # 
    # Initialize a counter
    counter = 0.
    # 
    # Preallocate a 2D numpy array, for speed
    reviewFeatureVecs = np.zeros((len(reviews),num_features),dtype="float32")
    # 
    # Loop through the reviews
    for review in reviews:
       #
       # Print a status message every 1000th review
       if counter%1000. == 0.:
           print "Review %d of %d" % (counter, len(reviews))
       # 
       # Call the function (defined above) that makes average feature vectors
       reviewFeatureVecs[counter] = makeFeatureVec(review, model, \
           num_features)
       #
       # Increment the counter
       counter = counter + 1.
    return reviewFeatureVecs

In [17]:
# ****************************************************************
# Calculate average feature vectors for training and testing sets,
# using the functions we defined above. Notice that we now use stop word
# removal.

clean_train_reviews = []
for review in df_data["review_text"]:
    clean_train_reviews.append( lowercase_remove_punctuation_and_numbers_and_tokenize_and_filter_more_stopwords_and_stem( review))

%time trainDataVecs = getAvgFeatureVecs( clean_train_reviews, model, num_features )

Review 0 of 219285


C:\Anaconda\lib\site-packages\ipykernel\__main__.py:44: DeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


Review 1000 of 219285
Review 2000 of 219285
Review 3000 of 219285
Review 4000 of 219285
Review 5000 of 219285
Review 6000 of 219285
Review 7000 of 219285
Review 8000 of 219285
Review 9000 of 219285
Review 10000 of 219285
Review 11000 of 219285
Review 12000 of 219285
Review 13000 of 219285
Review 14000 of 219285
Review 15000 of 219285
Review 16000 of 219285
Review 17000 of 219285
Review 18000 of 219285
Review 19000 of 219285
Review 20000 of 219285
Review 21000 of 219285
Review 22000 of 219285
Review 23000 of 219285
Review 24000 of 219285
Review 25000 of 219285
Review 26000 of 219285
Review 27000 of 219285
Review 28000 of 219285
Review 29000 of 219285
Review 30000 of 219285
Review 31000 of 219285
Review 32000 of 219285
Review 33000 of 219285
Review 34000 of 219285
Review 35000 of 219285
Review 36000 of 219285
Review 37000 of 219285
Review 38000 of 219285
Review 39000 of 219285
Review 40000 of 219285
Review 41000 of 219285
Review 42000 of 219285
Review 43000 of 219285
Review 44000 of 2192

In [18]:
# b = np.array(df_data.review_stars.copy())
# b[(b == 1) | (b == 2) | (b == 3)] = 0
# b[(b == 4) | (b == 5)] = 1

In [19]:
# unique, counts = np.unique(b, return_counts=True)

# print np.asarray((unique, counts)).T

[[   0  706]
 [   1 1519]]


In [20]:
# trainDataVecs = np.column_stack((trainDataVecs, b))

In [18]:
trainDataVecs

array([[-0.01919049,  0.00020651,  0.07834566, ...,  0.02025122,
         0.01151156, -0.01196238],
       [ 0.01497182,  0.00740789,  0.00292345, ...,  0.00717593,
         0.01075724, -0.00030977],
       [-0.00480012,  0.00025306,  0.00292017, ..., -0.01008894,
        -0.00847206, -0.00038908],
       ..., 
       [ 0.02289628, -0.01005408,  0.00365694, ...,  0.0110136 ,
         0.00381733, -0.01780435],
       [-0.01625548,  0.01062582, -0.00720449, ...,  0.0120312 ,
         0.02036202, -0.00255128],
       [-0.01710371, -0.00243994,  0.02431394, ...,  0.01426501,
         0.01236991, -0.00726273]], dtype=float32)

In [19]:
trainDataVecs.shape

(219285L, 300L)

In [20]:
word2vec_feature_matrix_file = os.path.join(YELP_DATA_WORD_2_VEC_MODEL_DIR, 'word2vec_feature_matrix'+data_subset+'.csv')
np.savetxt(word2vec_feature_matrix_file, trainDataVecs, delimiter=",")

In [21]:
test = np.genfromtxt(
    word2vec_feature_matrix_file,           # file name
    delimiter=',')           # column delimiter                                

In [22]:
np.array_equal(trainDataVecs, test)

True